# Notebook 3: Transformação e Limpeza de Dados da Tabela Indication

## Introdução
Neste notebook, iremos implementar funções para transformar e limpar os dados da tabela Indication do conjunto de dados FAERS. Com base na análise exploratória realizada no notebook anterior, identificámos várias áreas que requerem atenção, incluindo reclassificação de indicações desconhecidas e tratamento de valores nulos.

## Objetivos
- Criar funções reutilizáveis para limpeza e transformação de dados.
- Reclassificar indicações desconhecidas.
- Preparar os dados para análises futuras.

## Estrutura do Notebook
1. Importação de Bibliotecas
2. Carregamento dos Dados
3. Definição Manual do Schema e Renomeação das Colunas
4. Transformações e Limpeza
   - Tratamento de Valores Nulos
   - Reclassificação das Indicações
5. Função Principal de Transformação
6. Validação dos Dados Transformados
7. Guardar os Dados Transformados


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, when, to_date

In [0]:
# Carregar dados do Parquet
df_indication = spark.read.parquet('dbfs:/FileStore/FAERS-grupo-4/indication_raw')

In [0]:
# Definir metadados atualizados
meta_data_indication = {
    "primaryid": "Unique number for identifying a FAERS report.",
    "caseid": "Number for identifying a FAERS case.",
    "indi_drug_seq": "Drug sequence number for identifying a drug for a Case.",
    "indi_pt": "Preferred Term-level medical terminology describing the Indication for use using the Medical Dictionary for Regulatory Activities (MedDRA).",
    "indication_type": "Type of indication, either 'Unknown' or 'Known'."
}

# Renomear colunas para corresponder aos metadados
df_indication = df_indication.withColumnRenamed("primaryid", "primary_id") \
                             .withColumnRenamed("caseid", "caseid") \
                             .withColumnRenamed("indi_drug_seq", "indi_drug_seq") \
                             .withColumnRenamed("indi_pt", "indi_pt")

# Exibir o schema atualizado
df_indication.printSchema()

root
 |-- primary_id: long (nullable = true)
 |-- caseid: integer (nullable = true)
 |-- indi_drug_seq: integer (nullable = true)
 |-- indi_pt: string (nullable = true)



## Funções de transformação dos dados

In [0]:
# Função para preencher valores nulos
def preencher_valores_nulos(df):
    return df.fillna({
        "indi_drug_seq": -1,  # Usar -1 para indicar valores nulos originalmente
        "indi_pt": "N/A"  # Usar "N/A" se "Unknown" tiver um significado específico
    })

# Função para identificar tipos de indicações
def identificar_tipo_indicacao(df):
    return df.withColumn(
        "indication_type",
        when(col("indi_pt") == "Product used for unknown indication", "Unknown")
         .otherwise("Known")
    )

In [0]:
# Função principal de transformação
def transformar_dados_indication(df):
    df = preencher_valores_nulos(df)
    df = identificar_tipo_indicacao(df)
    return df

# Aplicar a função de transformação
df_indication_transformada = transformar_dados_indication(df_indication)

# Função para validar os dados transformados
def validar_dados_transformados(df):
    total_registos = df.count()
    print(f"Número total de registos após as alterações: {total_registos}")
    df.printSchema()
    df.show(5)

# Validar os dados transformados
validar_dados_transformados(df_indication_transformada)

# Função para gravar dados
def gravar_dados(df, caminho):
    df.write.mode("overwrite").parquet(caminho)

# Aplicar os metadados
print("Metadados da tabela indication:")
for column, description in meta_data_indication.items():
    if column in df_indication_transformada.columns:
            df_indication_transformada = df_indication_transformada.withMetadata(column, {'description':description})
    print(f"- {column}: {description}")

# Especificar o caminho para salvar o DataFrame transformado
save_path = "dbfs:/FileStore/FAERS-grupo-4/indication_final"

# Gravar o DataFrame transformado
gravar_dados(df_indication_transformada, save_path)



Número total de registos após as alterações: 5843230
root
 |-- primary_id: long (nullable = true)
 |-- caseid: integer (nullable = true)
 |-- indi_drug_seq: integer (nullable = false)
 |-- indi_pt: string (nullable = false)
 |-- indication_type: string (nullable = false)

+----------+--------+-------------+--------------------+---------------+
|primary_id|  caseid|indi_drug_seq|             indi_pt|indication_type|
+----------+--------+-------------+--------------------+---------------+
| 216766471|21676647|            4|Product used for ...|        Unknown|
| 216766481|21676648|            1|Product used for ...|        Unknown|
| 216766491|21676649|            1|Product used for ...|        Unknown|
| 216766501|21676650|            1|  Colitis ulcerative|          Known|
| 216766511|21676651|            1|Rheumatoid arthritis|          Known|
+----------+--------+-------------+--------------------+---------------+
only showing top 5 rows

Metadados da tabela indication:
- primaryid: 